In [ ]:
# np.set_printoptions(edgeitems=30, linewidth=100000) #Esto es para ajustar el ancho de la celda 
    # formatter=dict(float=lambda x: "%.10g" % x)) #y esto para la cantidad de decimales que queres que te imprima (no recomiendo)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import cv2
from scipy.stats import stats
import matplotlib.image as mpimg
import time
import copy as cp
import math

def norm2(v):
  suma = 0
  for i in range(v.size):
    suma += v[i]**2
  return float(suma)

def sortear_en_esfera(dimension):
  ret = [0 for _ in range(dimension)]
  for i in range(dimension):
    ret[i] = np.random.normal()
  ret = np.array(ret)
  ret /= np.linalg.norm(ret)
  return ret

def proyeccion_ortogonal(v,w): #proyecto v al complemento ortogonal de w
  v2 = v-w*np.dot(v,w)
  normaa = np.linalg.norm(v2)
  if normaa == 0:
    return v2
  return v2/np.linalg.norm(v2)

class EigenGame:
  def __init__(self,n,k,components,expected_components,ruido):
    self.n = n #dimension de las muestras
    self.k = k #cantidad de muestras
    self.components = components #componentes principales reales de mi espacio
    self.expected_components = expected_components #cantidad de componentes que quiero hallar
    self.ruido = ruido #ruido que le pongo a las muestas
    self.tiempos = [] #cuanto tiempo tarde en calcular cada componente
    self.iteraciones = [] #cuantas iteraciones tarde en calcular cada componente
    self.id = np.eye(n)
    self.matrices_proyeccion = []
    assert(components >= expected_components)

    dir = [sortear_en_esfera(n) for _ in range(components)]
    X = np.zeros((k,n))
    weights = np.random.normal(size = (k,components)) #sorteo cuanto de cada componente va a tener cada muestra
    for i in range(k):
      for j in range(components):
        X[i] += dir[j]*weights[i][j]
      X[i] += ruido*sortear_en_esfera(n) #le agrego una componente de ruido aleatoria
    # X = X - np.mean(X.T, axis=1) #esta linea hace que las columnas de X tengan promedio 0
    self.dir = dir
    self.M = np.matmul(X.T,X) #shape (n,n)
    self.X = X #shape (k,n)

    self.vectors = np.eye(n)
    self.Mv = [np.dot(self.M,self.vectors[i]) for i in range(n)] #vectores de n filas
    self.vtMv = [np.dot(self.vectors[i].T,self.Mv[i]) for i in range(expected_components)]
    self.vMv = np.array([[np.dot(self.vectors[i],self.Mv[j]) for j in range(expected_components)] for i in range(expected_components)])

  def set_M_y_vectores_iniciales(self,M, v = 0):
    if v == 0:
      self.vectors = np.eye(self.n)
    else :
      self.vectors = v
    self.M = M
    self.Mv = [np.dot(self.M,self.vectors[i]) for i in range(self.n)] #vectores de n filas
    self.vtMv = [np.dot(self.vectors[i].T,self.Mv[i]) for i in range(self.expected_components)]
    self.vMv = np.array([[np.dot(self.vectors[i],self.Mv[j]) for j in range(self.expected_components)] for i in range(self.expected_components)])

  def util(self, vector, i):
    Mvector = np.dot(self.M,vector)
    answer = np.dot(vector.T,Mvector)
    for j in range(i):
      answer -= (np.dot(vector.T,self.Mv[j])**2)/self.vtMv[j]
    return answer

  def evolutivo(self,T,error):
    start_time = time.time()
    for i in range(self.expected_components):
      for kk in range(i): #lineas agregadas
        self.vectors[i] = proyeccion_ortogonal(self.vectors[i],self.vectors[kk])
      step = 32
      for t in range(T):
        if step < error:
          print(i+1, t+1,  time.time() - start_time, (time.time() - start_time)/(t+1))
          self.tiempos.append(time.time() - start_time)
          self.iteraciones.append(t)
          start_time = time.time()
          break
        candidato = self.vectors[i]
        utilidad = self.util(candidato,i)
        unmodified = True
        for jj in range(2*self.n):
          j = jj//2
          sumo = True if jj%2==0 else False
          nuevo = self.vectors[i]+step*self.id[j] if sumo else self.vectors[i]-step*self.id[j]
          nuevo /= np.linalg.norm(nuevo)
          for kk in range(i):
            nuevo = proyeccion_ortogonal(nuevo,self.vectors[kk])
          if np.linalg.norm(nuevo) == 0: #
            continue #
          # print("ahreee", np.linalg.norm(nuevo),nuevo)
          assert(math.isclose(np.linalg.norm(nuevo),1))
          utilidad_nueva = self.util(nuevo,i)
          if utilidad_nueva < utilidad: #aca hubo cambio
            candidato = nuevo
            utilidad = utilidad_nueva
            unmodified = False
        if unmodified :
          step /= 2
        else:
          self.vectors[i] = candidato
          self.Mv[i] = np.dot(self.M,self.vectors[i])
          self.vtMv[i] = np.dot(self.vectors[i].T,self.Mv[i])
        if t == T-1:
          print("ATENCION ATENCION ATENCION", i+1, t+1,  time.time() - start_time, (time.time() - start_time)/(t+1))
          self.tiempos.append(time.time() - start_time)
          self.iteraciones.append(t)
          start_time = time.time()
    return self.vectors[0:self.expected_components]
  

In [ ]:
np.set_printoptions(edgeitems=30, linewidth=100000) #Esto es para ajustar el ancho de la celda

def kill_zeros(matrix):
  copy = matrix
  for i in range(len(matrix)):
    for j in range(len(matrix)):
      if abs(copy[i][j]) < 1e-9:
        copy[i][j] = 0
  return copy

# enes = [200]#, 300, 500]
enes = [50]
respuestas = []
cant_de_componentes = 8
if True:
  for n in enes:
    V = [(0.06 if i/n < 0.35 else 240) for i in range(n)]
    V = [0 for i in range(n)]
    eg = EigenGame(n,n,cant_de_componentes,cant_de_componentes,1)
    MM = np.zeros((n,n))
    for i in range(n):
      MM[i][i] = 2*n*n - V[i]
      if i > 0 :
        MM[i][i-1] = -1*n*n
      if i <n-1:
        MM[i][i+1] = -1*n*n
    eg.set_M_y_vectores_iniciales(MM)
    print(np.linalg.eigvalsh(MM).tolist()[0:8])
    v = eg.evolutivo(200000,1e-7)
    respuestas.append(eg.iteraciones)
    print(kill_zeros(np.matmul(v,v.T)))
    print(kill_zeros(np.matmul(v,np.matmul(eg.M,v.T))))#/(eg.n*eg.n*eg.k*eg.k)))
    print(v.tolist())
  print(respuestas)
  respuestas = []

[9.483356314780018, 37.89745164032113, 85.13450158049204, 151.01531982495436, 235.28999786421838, 337.6388529782223, 457.6736409023812, 594.9390285710788]
1 14202 38.55578303337097 0.0027148136873007527
2 6700 31.080538034439087 0.004638886416136329
3 10858 70.81690669059753 0.006522094968450423
4 9236 75.69847178459167 0.008196023492495081
5 7619 76.52827882766724 0.010044399561383215
6 6592 77.77266764640808 0.011798038427690857
7 5660 77.22412323951721 0.01364383819667695
8 4869 75.61496090888977 0.01552987505116975
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
[[ 9.48335635e+00 -9.92346909e-04 -9.05070973e-05 -2.55938106e-05 -7.88977852e-05  2.07770073e-04 -1.52915076e-04  2.14275073e-05]
 [-9.92346909e-04  3.78974516e+01 -4.69020989e-04 -5.33227616e-04 -3.47580939e-04  2.69384002e-04  1.08800367e-05 -1.32010126e-

In [ ]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000) #Esto es para ajustar el ancho de la celda

x = [9.48335635, 3.78974516e+01, 8.51345016e+01, 1.51015320e+02, 2.35289998e+02, 3.37638853e+02, 4.57673641e+02, 5.94939029e+02]
y = [9.67435491e+00, 3.86880568e+01, 8.70130410e+01, 1.54602553e+02, 2.41391205e+02, 3.47295036e+02, 4.72211589e+02, 6.16020016e+02]

# for n in [50,100]:
#   MM = np.zeros((n,n))
#   for i in range(n):
#     MM[i][i] = 2*n*n 
#     if i > 0 :
#       MM[i][i-1] = -1*n*n
#     if i <n-1:
#       MM[i][i+1] = -1*n*n
#   if n == 50:
#     x0 = np.linalg.eigvalsh(MM).tolist()[0:8]
#   else:
#     y0 = np.linalg.eigvalsh(MM).tolist()[0:8]
  # print(x if n==50 else y)
  # print(np.linalg.eigvalsh(MM).tolist()[0:8])

# for i in range(8):
#   print(abs(x[i]-x0[i]))
# print("==========")
# for i in range(8):
#   print(abs(y[i]-y0[i]))
# print("==========")

print("---------------------------")


for i in range(len(x)):
  x[i]=x[i]/((np.pi*(i+1))**2)
print(x)
for i in range(len(y)):
  y[i]=y[i]/((np.pi*(i+1))**2)
print(y)

NameError: ignored

In [ ]:
# Resultados de ejecuciones anteriores

# ===================== n = 50 =====================
# 1 14202 40.77105951309204 0.002870797120489011
# 2 6700 34.43948936462402 0.005140222435566917
# 3 10858 76.007000207901 0.007000092183013146
# 4 9236 83.13150119781494 0.009000812272790088
# 5 7619 84.72304439544678 0.011119969264464924
# 6 6592 83.56947112083435 0.012677407778293183
# 7 5660 82.8119330406189 0.014631083786698196
# 8 4869 81.86099910736084 0.016812692655139107
# [[1. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 1. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 1. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 1. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 1. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 1. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 1. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 1.]]
# [[ 9.48335635e+00 -9.92346909e-04 -9.05070973e-05 -2.55938106e-05 -7.88977852e-05  2.07770073e-04 -1.52915076e-04  2.14275073e-05]
#  [-9.92346909e-04  3.78974516e+01 -4.69020989e-04 -5.33227616e-04 -3.47580939e-04  2.69384002e-04  1.08800367e-05 -1.32010126e-04]
#  [-9.05070973e-05 -4.69020989e-04  8.51345016e+01  9.44750120e-04 -2.46911899e-05  3.50329650e-05 -9.97776894e-05 -1.25293900e-04]
#  [-2.55938106e-05 -5.33227616e-04  9.44750120e-04  1.51015320e+02 -9.82350696e-04  4.16342394e-05  1.91004337e-05 -5.36132674e-05]
#  [-7.88977852e-05 -3.47580939e-04 -2.46911900e-05 -9.82350696e-04  2.35289998e+02  9.24588767e-04 -2.16857222e-05  5.15529125e-05]
#  [ 2.07770073e-04  2.69384002e-04  3.50329650e-05  4.16342393e-05  9.24588767e-04  3.37638853e+02  8.96658876e-04  5.60293981e-06]
#  [-1.52915076e-04  1.08800366e-05 -9.97776893e-05  1.91004338e-05 -2.16857222e-05  8.96658876e-04  4.57673641e+02  8.16858919e-04]
#  [ 2.14275074e-05 -1.32010126e-04 -1.25293900e-04 -5.36132673e-05  5.15529125e-05  5.60293977e-06  8.16858919e-04  5.94939029e+02]]

# v = [[0.012189937535976166, 0.024333653107762605, 0.03638512420451285, 0.04829863668803738, 0.060028939228219225, 0.07153163485299933, 0.0827630238541944, 0.09368049846182681, 0.10424270737890255, 0.11440952631241563, 0.12414241767850864, 0.1334045007225455, 0.14216065214894913, 0.15037763666645448, 0.1580242403665799, 0.16507150750372143, 0.17149266714210576, 0.17726335500886967, 0.1823616033381327, 0.18676817640037946, 0.19046629730149248, 0.1934419886424782, 0.19568390777038433, 0.19718363360981467, 0.19793541536516615, 0.19793627557147717, 0.19718636865060593, 0.1956883675575461, 0.19344798621205075, 0.19047375448550116, 0.18677696155073872, 0.18237157955870348, 0.17727435148848722, 0.17150454464049492, 0.16508407346615284, 0.15803737319393424, 0.15039112487440873, 0.14217431477697715, 0.13341813229746527, 0.12415576238908864, 0.11442240752563593, 0.10425500320831722, 0.09369208985337392, 0.08277369841441574, 0.07154122816103343, 0.06003726551543601, 0.04830550308771619, 0.036390438560087945, 0.024337326998315426, 0.012191798897675202], [0.024335375514689637, 0.048301859687356806, 0.07153618368690007, 0.09368618978592815, 0.11441612559367587, 0.13341167500632994, 0.1503848953038678, 0.16507853726556954, 0.17726985776940482, 0.18677403437382653, 0.19344699405100796, 0.19718759710300252, 0.1979391468252775, 0.19569021527750058, 0.19047491890515783, 0.18237233344098575, 0.1715051262630568, 0.15803820551914635, 0.1421755481433403, 0.12415770807979368, 0.10425777789176664, 0.08277747325122639, 0.060042321307002565, 0.0363970590804404, 0.012200038327089486, -0.012181742530805966, -0.036378895153886155, -0.06002460642741992, -0.08276039297537242, -0.1042416199708826, -0.1241426353792291, -0.14216171353508636, -0.15802558525787183, -0.17149377153867784, -0.1823621784236824, -0.19046601443530628, -0.19568248191144635, -0.19793246582749066, -0.1971818253726207, -0.1934420222567559, -0.186769697102565, -0.1772661190869171, -0.16507542938394973, -0.15038230142801853, -0.1334095202707383, -0.1144143946556758, -0.09368489544014527, -0.07153523478295513, -0.04830121735737044, -0.024335017135259214], [0.03638890364308398, 0.07153852919350559, 0.10425190408919106, 0.13341506075350143, 0.15803492169201644, 0.17727300502645643, 0.1904742207140292, 0.19718904473196674, 0.1971888414211625, 0.190473571631123, 0.17727197905103917, 0.15803354366014144, 0.1334134935800481, 0.10425032721156295, 0.07153711495636349, 0.036387928985142114, -4.3674481913053084e-07, -0.036388666885780165, -0.07153768307275912, -0.10425045605460335, -0.1334129802506662, -0.15803221425390834, -0.17726971865045787, -0.19047040878527488, -0.19718473982875384, -0.19718418605478402, -0.19046879246249585, -0.17726730348763942, -0.15802928122832038, -0.13340983493915115, -0.10424734879026581, -0.07153492019006279, -0.03638658565838978, 9.067727748607434e-07, 0.036388236371978744, 0.07153631848093489, 0.10424821748377504, 0.1334100163538859, 0.1580286076296967, 0.1772657082434179, 0.19046628385521083, 0.19718077793528285, 0.19718057626989496, 0.1904656716000505, 0.17726470581020926, 0.15802729365034446, 0.13340851363634199, 0.10424668076418908, 0.07153486961430343, 0.03638697489522155], [0.04830243774366368, 0.09368702696677284, 0.1334122734777768, 0.16507853370175357, 0.1867729333547736, 0.19718498527701395, 0.19568575829533771, 0.1823658719029896, 0.1580298994154138, 0.12414781911740262, 0.08276638403626622, 0.036385323466295735, -0.012193579530297732, -0.06003584122547813, -0.10425147377299543, -0.1421696157620718, -0.17149977044863818, -0.1904701464108722, -0.19793480076190745, -0.19344302548430825, -0.1772659315761823, -0.15038076920575955, -0.11441171912147922, -0.07153152366905935, -0.024330381671130757, 0.024340468628894014, 0.07154112341823067, 0.11442023661957125, 0.15038769574006466, 0.1772707958561941, 0.1934456221836675, 0.19793509389276426, 0.19046809816670535, 0.1714957298131331, 0.14216402495157823, 0.10424478530393402, 0.06002856561496259, 0.012186287758104757, -0.03639222428444408, -0.08277249001550421, -0.12415285922835123, -0.15803369242987633, -0.18236852979248433, -0.19568734054304018, -0.19718564306387085, -0.18677292143301527, -0.16507807846478356, -0.1334116523252654, -0.09368642121126028, -0.048302078160735624], [-0.06003313780466829, -0.11441624465886104, -0.15803084631534894, -0.18677205253331908, -0.19793492002347934, -0.19046875186622597, -0.16507622775895794, -0.12414721530080347, -0.07153388447180296, -0.012188083702241595, 0.048304926140824045, 0.10425164374065225, 0.15038660737224993, 0.1823677521701955, 0.19718511655343637, 0.1934441661233948, 0.17149696841831097, 0.1334090685098299, 0.08276529752364774, 0.02433199703317755, -0.03639143532382486, -0.09368980003559825, -0.14217040469663506, -0.17727044430706743, -0.19568647001022735, -0.1956852057156259, -0.17726693403451457, -0.14216508142599996, -0.09368321924475587, -0.03638427260430108, 0.02433914008482772, 0.0827718490360685, 0.1334143890249197, 0.1715004573403975, 0.19344557539541724, 0.1971844850323791, 0.18236530051845834, 0.15038264269290885, 0.1042466377566784, 0.0482994099041687, -0.012193597404900485, -0.07153896166134187, -0.12415141325561006, -0.16507939495103258, -0.19047087450137357, -0.19793618884943984, -0.18677270417947972, -0.1580309706520998, -0.11441613531733567, -0.06003306517035024], [-0.07153671208040727, -0.13341188808447457, -0.1772689776940746, -0.19718475885594133, -0.19046953443882123, -0.15803018236712493, -0.10424795275482435, -0.0363864930518324, 0.03638920627994877, 0.10425029699038979, 0.15803173960084735, 0.1904699796507356, 0.19718403221489217, 0.17726709528224982, 0.13340917283991927, 0.07153357428778101, -3.069604799500883e-06, -0.07153931804026706, -0.1334138531023606, -0.17727010246160058, -0.19718503350267627, -0.19046903713635024, -0.15802911120645774, -0.10424647721544185, -0.0363848402073216, 0.036390781836882845, 0.1042517304455981, 0.15803293096979976, 0.19047085465238178, 0.19718458077586326, 0.17726749404500727, 0.1334095071351677, 0.07153384907074811, -2.859157966782788e-06, -0.07153920204226923, -0.13341374437122702, -0.17727001440382636, -0.19718513707413685, -0.19046934345109146, -0.1580297403827858, -0.10424746031771401, -0.036386063515181845, 0.03638950500241028, 0.10425047016686537, 0.1580319608777945, 0.19047045031207557, 0.1971850416628061, 0.17726881094981942, 0.13341160030233085, 0.07153646445889199], [-0.08276851437707143, -0.15038464320292622, -0.190469790955459, -0.19568557952622312, -0.16507718827073892, -0.10424808633438597, -0.024334204258339855, 0.06003447296535911, 0.13341258755828606, 0.18236691861430543, 0.19793536754528843, 0.17726775836378664, 0.12414778296899671, 0.048300172531399246, -0.036389775798407686, -0.11441788586759127, -0.1714994969193097, -0.19718470498359128, -0.18677134059431139, -0.1421658447233495, -0.07153409577512816, 0.012193413443143668, 0.09368873823187768, 0.15803252013368285, 0.1934453472675122, 0.19344427476197262, 0.1580293982996335, 0.0936841778325369, 0.012188265610405554, -0.07153892490852523, -0.14216949159063358, -0.18677323357665282, -0.19718457175105392, -0.17149754471291104, -0.11441450260286595, -0.03638571396797913, 0.048304203257687924, 0.12415117030830021, 0.17726986233388936, 0.19793596197151198, 0.18236616013985935, 0.13341082606255836, 0.06003215790651377, -0.024336514611203806, -0.1042499124326139, -0.16507843267432906, -0.19568629379261762, -0.1904700727733069, -0.15038468833697308, -0.08276847520383733], [-0.0936864577118173, -0.16507774489174062, -0.19718439333910975, -0.1823658278206754, -0.1241485768149293, -0.03638697048528814, 0.06003399691876413, 0.14216833208698088, 0.1904700324503053, 0.19344439358622612, 0.1503836375546034, 0.07153520973912934, -0.024336844969254815, -0.11441731863843338, -0.17726928490291433, -0.1979353971477488, -0.17149766479356648, -0.1042475861163702, -0.012189040891819969, 0.08277022183287859, 0.1580321414859986, 0.19568620125000824, 0.18677168133338048, 0.13341008100882606, 0.04830010533951478, -0.04830415244585689, -0.13341324430973342, -0.18677325720574506, -0.1956857956379537, -0.15802987309302421, -0.082766734737373, 0.012192829427477493, 0.10425076606594166, 0.17149958409180477, 0.19793573330469855, 0.1772680904564186, 0.11441498840270452, 0.024333970994834458, -0.07153790157619232, -0.15038564619270536, -0.1934453086996005, -0.19046979889370555, -0.14216720386434487, -0.06003233084389457, 0.03638870517397584, 0.12415009810610617, 0.18236689916518353, 0.19718490555561033, 0.165077871318247, 0.09368643110778456]]

# ========================= n = 100 ==========================
# 1 120847 1203.8520820140839 0.009961787075635635
# 2 46334 793.6681990623474 0.01712928303739026
# 3 69550 1663.9950115680695 0.023925161934173166
# 4 60539 1866.6480786800385 0.03083381092968386
# 5 51735 1946.3909430503845 0.03762232422932365
# 6 46038 2051.335968732834 0.044557451880761725
# 7 40164 2090.6848709583282 0.05205370161521721
# 8 35320 2113.580859899521 0.05984090773867589
# [[1. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 1. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 1. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 1. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 1. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 1. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 1. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 1.]]
# [[ 9.67435491e+00  4.64199630e-03 -4.67392854e-05  1.69760796e-07  6.29609472e-05  1.39912414e-03 -1.02557993e-04  4.46786208e-05]
#  [ 4.64199630e-03  3.86880568e+01  8.84938883e-05 -4.73910194e-03  1.99211322e-04  6.02671473e-04 -7.84635756e-04 -5.94245418e-05]
#  [-4.67392855e-05  8.84938884e-05  8.70130410e+01  5.13670895e-03 -3.50046716e-04 -4.98262916e-04 -1.81659834e-04  6.97546219e-04]
#  [ 1.69760743e-07 -4.73910194e-03  5.13670895e-03  1.54602553e+02  5.50500424e-03  6.97386305e-04  2.98217577e-04  4.36186333e-04]
#  [ 6.29609471e-05  1.99211322e-04 -3.50046716e-04  5.50500424e-03  2.41391205e+02  5.17025724e-03  2.00834034e-05 -2.72940557e-04]
#  [ 1.39912414e-03  6.02671473e-04 -4.98262917e-04  6.97386305e-04  5.17025724e-03  3.47295036e+02 -5.04805072e-03  3.83574595e-04]
#  [-1.02557993e-04 -7.84635757e-04 -1.81659834e-04  2.98217577e-04  2.00834033e-05 -5.04805072e-03  4.72211589e+02  5.45763833e-03]
#  [ 4.46786209e-05 -5.94245417e-05  6.97546219e-04  4.36186333e-04 -2.72940557e-04  3.83574595e-04  5.45763833e-03  6.16020016e+02]]

# v = [[0.0043780082138794585, 0.008751705635674144, 0.013116823457622388, 0.01746919449512998, 0.021804719554474473, 0.02611911808366902, 0.030408181215552648, 0.0346677197602165, 0.03889361534091856, 0.043081836769799646, 0.047228347583844595, 0.05132916528611211, 0.05538029091246548, 0.05937781704526329, 0.06331778008695954, 0.06719641921152032, 0.07101001257151929, 0.07475485195422533, 0.07842730529575952, 0.08202385553689295, 0.08554102134380484, 0.08897538241623268, 0.09232360356878112, 0.0955824765320556, 0.0987488696501364, 0.10181962683216374, 0.10479184992395006, 0.10766268405038024, 0.1104292632202056, 0.11308895264147449, 0.11563913497816543, 0.11807741314258963, 0.12040145056632852, 0.12260890883955512, 0.12469767113175187, 0.12666573011415416, 0.12851121568632123, 0.13023234081786084, 0.13182741042455676, 0.13329490124563792, 0.13463338335998498, 0.1358416022039615, 0.13691839365119038, 0.13786270431746814, 0.13867357316489684, 0.13935025046828092, 0.1398920145888612, 0.14029841464842707, 0.14056904524892133, 0.1407035806969768, 0.14070190005527836, 0.1405641377772941, 0.14029047668880454, 0.13988113081243014, 0.13933650241989096, 0.13865716571365036, 0.13784368730552646, 0.1368969125082246, 0.13581778666232813, 0.13460730138353733, 0.1332666056651988, 0.13179706122744556, 0.13020010244065008, 0.12847719391849527, 0.12663010134264682, 0.1246605273784167, 0.12257041771467606, 0.12036179801736378, 0.11803675182293487, 0.11559755812605682, 0.11304660208904861, 0.11038629607287295, 0.10761921416944698, 0.10474803823584197, 0.10177560451773168, 0.09870479090748778, 0.0955385542902209, 0.09227990459621324, 0.0889320929027907, 0.0854983471138115, 0.08198191912572503, 0.07838619445414409, 0.07471466714164555, 0.07097097485236878, 0.06715863985639925, 0.06328135431638204, 0.05934287396512348, 0.055347095057041044, 0.051297806731036115, 0.047198973939817, 0.04305452773836752, 0.038868480747129376, 0.03464491947863612, 0.030387897896181657, 0.026101481290603176, 0.021789858396345715, 0.017457157722661345, 0.013107679974095643, 0.008745533299795823, 0.00437493927259638], [0.0087470168567207, 0.01746023867138045, 0.02610601145418856, 0.034650884037100545, 0.04306172125389177, 0.05130598939202744, 0.05935181709341988, 0.06716807511766774, 0.0747243938188425, 0.08199161924437808, 0.0889416676792162, 0.0955476134593579, 0.10178390548105871, 0.1076264997665868, 0.11305269532946384, 0.11804149729454795, 0.1225735999515264, 0.12663157343234288, 0.13019963098127982, 0.13326401706067256, 0.1358128287574448, 0.13783620037490585, 0.13932634666517013, 0.1402774821315615, 0.14068593850205363, 0.14055016512653778, 0.13987061684197405, 0.13864988954706023, 0.1368926050295226, 0.1346055776154607, 0.13179762777993598, 0.12847966413701015, 0.12466451177045597, 0.12036701577852359, 0.11560373989071333, 0.1103930834079226, 0.10475526358874428, 0.09871207347117095, 0.0922868581701767, 0.08550443170588298, 0.07839105181402313, 0.0709742439928173, 0.06328270604514415, 0.05534617025204121, 0.047195381529090356, 0.03886187105035117, 0.030377890251608756, 0.02177629264218019, 0.013090370206836408, 0.004353657234090427, -0.00439988505295418, -0.013136402731431605, -0.021822123734330633, -0.03042340694241346, -0.038907014199950865, -0.047240116760378675, -0.0553905164003618, -0.06332659844728968, -0.07101763600123855, -0.07843388879527224, -0.0855467157536785, -0.0923286151318198, -0.09875335162679097, -0.10479599331125079, -0.11043325319042915, -0.11564332148717395, -0.12040608256100822, -0.12470299533638479, -0.12851742513429232, -0.13183466716153147, -0.13464184657393363, -0.1369281189795399, -0.13868464618348417, -0.13990460303474092, -0.14058327635185083, -0.14071808291417806, -0.14030848618317313, -0.13935614799203722, -0.13786475737358883, -0.13584011412123592, -0.13328998935030345, -0.13022426491411573, -0.12665484235329152, -0.12259553540554112, -0.11806208162125359, -0.11307199831415292, -0.1076445823857348, -0.10180075700910052, -0.09556312505622559, -0.0889558214845944, -0.08200441280239361, -0.0747358629703589, -0.06717822219494356, -0.059360734302791776, -0.05131360593570354, -0.04306803573398421, -0.034655951003482115, -0.026109855783693466, -0.017462828113265425, -0.008748329366554685], [0.01311345786565136, 0.026112717176023353, 0.038884756346450465, 0.05131839889705363, 0.06330542828651003, 0.07474157453513998, 0.0855273534797334, 0.09556883796603076, 0.1047787402749003, 0.11307684097982769, 0.1203911047935445, 0.12665774804619218, 0.13182225264031466, 0.1358396985081287, 0.13867507495735854, 0.14030373364709284, 0.1407114782481085, 0.13989482433714168, 0.13786078816472605, 0.1346270846107923, 0.1302219322266357, 0.12468364787473664, 0.11806035292447507, 0.1104097367322563, 0.10179840359736748, 0.09230123037867838, 0.08200088138605369, 0.07098702890249196, 0.0593555646054437, 0.0472077215189614, 0.034649131000425874, 0.021789117638087843, 0.008739610973034025, -0.0043858855954896185, -0.017473129177773012, -0.03040826472319671, -0.04307879196462279, -0.05537440214382475, -0.06718815163369278, -0.07841718703105872, -0.08896382901937597, -0.09873634945370066, -0.10764972099874401, -0.11562629946161285, -0.12259671180599745, -0.12850032334571979, -0.1332857909097432, -0.1369114261720102, -0.13934573187597205, -0.14056747074298848, -0.14056607664666668, -0.139341653479638, -0.13690470168894475, -0.13327647260421327, -0.12848859480117097, -0.12258279249829777, -0.11561036284603515, -0.10763205526414954, -0.09871725186264758, -0.0889435312144169, -0.07839600191245812, -0.06716641776611314, -0.055352471753716326, -0.0430569131170243, -0.030386796800033714, -0.017452386282628855, -0.004366221580648589, 0.008757904086497124, 0.02180580672169451, 0.03466394220330738, 0.04722036894275078, 0.059365834388903825, 0.07099472039056737, 0.08200581403426546, 0.09230330781625287, 0.10179770702238708, 0.1104062550392107, 0.11805404815074072, 0.12467470809641588, 0.13021063999929383, 0.13461366576675396, 0.13784544669011345, 0.13987789352625668, 0.14069330363617077, 0.14028456963854966, 0.1386552577672577, 0.13581944879354124, 0.13180186171824343, 0.12663751416873034, 0.12037131017842946, 0.11305780730316295, 0.10476063227052652, 0.0955519458282411, 0.08551190972353351, 0.07472788416550284, 0.06329364379395337, 0.05130869081678577, 0.03887729911093428, 0.026107657455292102, 0.013110863876044964], [0.017463993758718068, 0.034657873261990706, 0.051315915400768165, 0.06718054709130573, 0.08200649948684305, 0.09556455130487844, 0.10764503828184938, 0.11806119852208398, 0.12665203563219074, 0.13328473568007368, 0.1378567056525289, 0.1402972543130058, 0.1405687095667482, 0.13866688665605778, 0.13462111439658142, 0.12849402348107494, 0.12038032808636788, 0.11040549245324495, 0.09872366580081132, 0.08551556936666603, 0.07098526709686308, 0.0553574794154609, 0.038873765869754655, 0.021789135456541285, 0.004367757865184988, -0.013121047297381123, -0.030406867616073262, -0.047222469001736905, -0.06330788256358709, -0.07841441837416557, -0.09230852554028382, -0.10477536556032387, -0.11562221798338902, -0.12468139432959474, -0.13181289362020454, -0.13690643571657019, -0.13988321660761074, -0.14069725666725327, -0.13933595876385282, -0.13582040666687553, -0.13020502033709422, -0.12257648140588229, -0.11305276659131908, -0.10178122035812272, -0.08893612929548524, -0.07471606405524923, -0.05934086929632406, -0.043048246255977245, -0.026090070809831112, -0.008728558842455745, 0.008767888189443155, 0.026128803782524416, 0.04308572887491439, 0.05937651620330387, 0.07474932886610237, 0.08896659343558212, 0.10180843582482071, 0.11307630883162707, 0.12259590664295208, 0.13022008964240553, 0.13583104056840414, 0.13934203452512006, 0.14069881633321887, 0.13988042337443693, 0.1368994993584183, 0.1318021061108158, 0.12466700700254185, 0.11560457321043614, 0.10475482729143765, 0.09228549751182942, 0.07838940471493505, 0.06328134756107602, 0.0471948595009944, 0.030378727299126156, 0.01309284948559017, -0.004395494488534736, -0.021815962922225932, -0.03889919115720536, -0.05538111371680317, -0.0710068582581953, -0.08553500710339751, -0.09874082096436439, -0.11042023522360249, -0.12039276018891933, -0.12850422578251489, -0.1346291747246522, -0.1386728517661099, -0.14057277638589036, -0.14029961850342976, -0.1378575956295907, -0.13328444040856025, -0.12665084392149212, -0.1180592980559916, -0.10764265156610195, -0.09556190560113262, -0.08200383867548093, -0.06717807912307319, -0.05131381340872123, -0.03465632652898439, -0.017463131154385855], [0.021797395376784604, 0.04306856294545954, 0.06329998606072464, 0.08200327107035545, 0.09872712304943555, 0.11306770309844238, 0.1246788389229941, 0.13328022556034788, 0.13866420909723592, 0.14070079114625472, 0.1393408992330824, 0.13461728049759744, 0.12664400401907844, 0.11561347833902837, 0.10179197401022982, 0.08551318272531162, 0.06717005170209876, 0.04720535480257529, 0.026101188959905926, 0.004366983617509424, -0.017472594600452248, -0.03889034059993256, -0.05936923182469885, -0.07841494282432866, -0.09556777503218303, -0.11041368889887651, -0.12259421159347708, -0.13181531353250614, -0.1378544545637473, -0.14056576447446578, -0.13988387516303452, -0.13582519440515423, -0.12848772963198374, -0.1180485811020459, -0.10475985984930773, -0.08894234698413157, -0.07097781464039851, -0.05129992222602944, -0.030383657029279715, -0.008733905082473452, 0.01312668510288207, 0.03467036918340719, 0.05537717933946671, 0.07474724972775224, 0.09231296590215994, 0.1076503518540255, 0.120389100480754, 0.1302217393818415, 0.13691088753787278, 0.14029506760423596, 0.14029260794888063, 0.1369036979627778, 0.13021005556027163, 0.12037329460194063, 0.1076308829129345, 0.09229032201784541, 0.07472197433497078, 0.05534993139907555, 0.0346416398694817, 0.013097154559476572, -0.008763485896663559, -0.03041263043917131, -0.05132760242209339, -0.07100356545100991, -0.08896563222816482, -0.10478021510032823, -0.11806548963538435, -0.12850078746388371, -0.13583418890439125, -0.13988882716484594, -0.14056680272909086, -0.13785167558430003, -0.1318090705988603, -0.12258482496683207, -0.11040154266290003, -0.09555327424034095, -0.07839849311188843, -0.05935128148159325, -0.03887144150103396, -0.01745330571056202, 0.0043861272560291615, 0.026119680523383437, 0.047222756115391715, 0.06718604143240497, 0.08552748338459606, 0.10180446310026828, 0.11562414742048363, 0.1266529072381676, 0.13462440154683816, 0.1393462779417757, 0.1407046233038817, 0.1386666570805892, 0.13328159066352857, 0.12467935640575835, 0.11306755274007421, 0.09872653658475966, 0.08200250046036042, 0.06329913236252555, 0.04306782994956386, 0.021796984884760826], [0.026110279776100247, 0.05131368476086777, 0.07473490641435274, 0.09556057344140811, 0.11306735799462024, 0.1266472772772988, 0.13582871604260885, 0.14029274940466474, 0.13988434099050312, 0.13461765031300943, 0.1246755957794255, 0.11040344839581027, 0.0922969489561457, 0.07098492361874963, 0.04720749209939767, 0.021790420544681718, -0.004383469472465235, -0.030405074909607453, -0.05537066270792457, -0.07841317284112408, -0.09873237295105987, -0.115622600191348, -0.12849723773768873, -0.13690914585174474, -0.14056619766113337, -0.13934149319192113, -0.1332774589386574, -0.12258463790988429, -0.10763435256686797, -0.08894593565118405, -0.06716846809044603, -0.043058332213455876, -0.017452780587859654, 0.00875889545884728, 0.034666337680589916, 0.059369827432859325, 0.08201143719963398, 0.10180481987624139, 0.11806253776239467, 0.13021988764630096, 0.13785472625098877, 0.14070195786999917, 0.13866260246771014, 0.13180744737323957, 0.12037465958978878, 0.10476127229410542, 0.08550950793944341, 0.06328806162154478, 0.03886854814985054, 0.013099122871309456, -0.013125305825871597, -0.03889393237367404, -0.06331185856355649, -0.08553100605605472, -0.10477976872482549, -0.12038956319515096, -0.131818300346175, -0.13866908197586053, -0.1407040842819273, -0.13785244067116184, -0.1302133187366549, -0.11805207647012128, -0.10179097831540351, -0.08199479363215058, -0.05935106316553223, -0.03464616051168941, -0.00873804510503495, 0.01747346557138943, 0.04307809622748726, 0.06718665957227583, 0.08896190903489593, 0.10764760915664029, 0.12259481666224313, 0.13328451688907766, 0.1393453902162366, 0.14056698157697464, 0.13690684867815872, 0.12849205869567287, 0.11561489750435781, 0.09872257738713047, 0.07840171675371159, 0.0553580970946715, 0.030391980684526313, 0.004370334338212091, -0.02180302157555081, -0.04721915283961882, -0.07099535103031976, -0.09230588308358022, -0.11041078584058682, -0.12468132056346308, -0.13462181295545753, -0.13988705108038818, -0.14029422735316033, -0.13582919704889182, -0.12664704550385014, -0.11306659467509612, -0.09555951500874971, -0.07473385122451644, -0.051312782239008384, -0.026109769374176268], [-0.03039836771582873, -0.05936118570702911, -0.08552077248215366, -0.10764181965941581, -0.12467973465254341, -0.13582997076107917, -0.14056605992455026, -0.1386643576709021, -0.13021463069747102, -0.11561595958714133, -0.09555767342204653, -0.0709869079241467, -0.043063890686067675, -0.013107323219723356, 0.01746813648703316, 0.04721871104978898, 0.07473963556702792, 0.09873119607987935, 0.11806046355518952, 0.13181471426573463, 0.1393443883918536, 0.14029390548305734, 0.13461846080244966, 0.12258608691479868, 0.10476499404375174, 0.08199670322363764, 0.05535639088925773, 0.026102071492664743, -0.004384844934275462, -0.03466463333369263, -0.06330747085645924, -0.08896079053846022, -0.11041322582315062, -0.12665170249545626, -0.13690951019858524, -0.14070219210243848, -0.13785061448231659, -0.12848947225769927, -0.11306083732475768, -0.09229333801165379, -0.06716764860870912, -0.03887024519154543, -0.008737369007735859, 0.021808123359260644, 0.051323855292571295, 0.07841597177068985, 0.10180520754038065, 0.12038706661915004, 0.13328403652303, 0.1398872449639036, 0.13988477967530794, 0.13327685857685231, 0.1203755045559574, 0.10178988392472353, 0.07839764112002964, 0.051303426369203084, 0.0217865620942425, -0.008759117218440984, -0.038891272232325975, -0.06718695246712275, -0.09231003312242084, -0.11307408885302583, -0.12849875203746702, -0.13785564774241102, -0.14070280866530702, -0.13690589745914766, -0.12664417929910868, -0.11040230542940219, -0.08894721294798093, -0.06329194679297644, -0.0346480550774874, -0.00436806440716503, 0.02611815993710875, 0.05537101144555099, 0.0820091979125133, 0.10477490204150426, 0.12259315183638379, 0.13462252887668447, 0.1402949826391179, 0.13934268051311097, 0.1318105424453614, 0.11805417496315056, 0.09872320553107503, 0.07473052998928775, 0.04720905871418203, 0.01745836533342929, -0.013116704914494872, -0.04307240794627278, -0.07099427299826924, -0.09556379129702285, -0.11562065648811892, -0.13021792178228553, -0.13866626617373362, -0.14056671428971065, -0.13582958799829414, -0.12467859405749038, -0.10764025672021192, -0.08551917707471662, -0.05935988476096312, -0.030397634277832562], [-0.03465650608669117, -0.06717803714112414, -0.09556122239172075, -0.11805754491844976, -0.13328119761751883, -0.14029430598920903, -0.13866486642810844, -0.12849322006544267, -0.1104060144182323, -0.08551746856410807, -0.055360743146196906, -0.02179369133055685, 0.013115942006666345, 0.04721751941930383, 0.07841028107159569, 0.10477275210170424, 0.12468086002107055, 0.13690828525447832, 0.14070174931016385, 0.13582757431387973, 0.1225860051352148, 0.10179277258518422, 0.07472875611227923, 0.043061300291746235, 0.008741194781779467, -0.026117397604764607, -0.05936705454655524, -0.08895951802296743, -0.11307187190716432, -0.1302187477313387, -0.13934381657382935, -0.1398850752041757, -0.1318089938676987, -0.1156131437632284, -0.09229513798857156, -0.06329147555987522, -0.030388923111751554, 0.004385667654967052, 0.038890070379767244, 0.0709987629538267, 0.09873379367498168, 0.12038659559667307, 0.13462333823877615, 0.14056698680216323, 0.1378513393400852, 0.12664382740945518, 0.1076346752694462, 0.08199503487235933, 0.05130434833823778, 0.01745322767600543, -0.01747300740795126, -0.05132289793209878, -0.08201115738354046, -0.10764731641054637, -0.12665213144211077, -0.13785489742546841, -0.1405655650987114, -0.13461720716918563, -0.12037621652261385, -0.098719868182451, -0.07098221272178518, -0.0388719510859872, -0.004367046845403171, 0.03040687211219957, 0.06330779052186179, 0.09230879144353969, 0.11562333867246376, 0.13181535370377911, 0.13988737301861107, 0.1393420130831524, 0.13021303999642211, 0.11306277202923712, 0.08894770939406371, 0.059353331337333684, 0.026102636302384654, -0.008756009643863088, -0.043075329382477365, -0.07474110926219983, -0.10180277869627964, -0.12259330462894608, -0.135831972091923, -0.140703226261011, -0.13690705908617098, -0.12467720894673212, -0.10476710137065544, -0.07840325363660465, -0.0472096566794506, -0.013107893795209354, 0.021801277628885264, 0.05536739809277618, 0.08552281994231195, 0.1104100489639594, 0.12849596291886475, 0.13866639006083026, 0.14029487415150033, 0.13328101939153722, 0.11805691997664569, 0.09556043021231227, 0.06717734279501192, 0.03465609941483823]]



# ========================= n = 100 pero con limite eps = 100000 ==========================
# ATENCION ATENCION ATENCION 1 100000 1008.0319232940674 0.010080319242477417
# 2 47232 830.5336306095123 0.01758413007710038
# 3 70567 1734.2305700778961 0.024575659597165133
# 4 63181 1985.1284577846527 0.03141970622083106
# 5 51075 1962.1842999458313 0.038417705348234694
# 6 45667 2069.6199316978455 0.04531981372471972
# 7 40311 2101.1811244487762 0.05212426199211213
# 8 36036 2140.1629087924957 0.05938958014165248
# [[1. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 1. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 1. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 1. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 1. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 1. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 1. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 1.]]
# [[ 9.67436244e+00  1.53585131e-02  1.69183164e-03 -3.87424966e-04 -1.66079820e-03  5.04432182e-03 -1.84245557e-04 -1.48555441e-03]
#  [ 1.53585131e-02  3.86880495e+01 -4.06947605e-04 -5.55246422e-03  5.56903588e-04  3.41472761e-04 -6.38333688e-04  3.12532683e-04]
#  [ 1.69183164e-03 -4.06947605e-04  8.70130410e+01  5.26831451e-03 -5.84139967e-06  2.03688832e-04 -6.41194665e-04 -2.61383266e-04]
#  [-3.87424966e-04 -5.55246422e-03  5.26831451e-03  1.54602552e+02  5.12252714e-03  2.43184497e-04  1.69817298e-05  2.26109670e-04]
#  [-1.66079820e-03  5.56903588e-04 -5.84139970e-06  5.12252714e-03  2.41391205e+02  5.69134908e-03 -3.49876517e-04  2.44581584e-04]
#  [ 5.04432182e-03  3.41472761e-04  2.03688832e-04  2.43184497e-04  5.69134908e-03  3.47295035e+02 -5.45244489e-03 -2.13988924e-04]
#  [-1.84245557e-04 -6.38333688e-04 -6.41194665e-04  1.69817298e-05 -3.49876517e-04 -5.45244489e-03  4.72211589e+02  5.22968678e-03]
#  [-1.48555441e-03  3.12532683e-04 -2.61383266e-04  2.26109669e-04  2.44581584e-04 -2.13988924e-04  5.22968678e-03  6.16020016e+02]]